In [80]:
import os
import requests
from datetime import datetime
import re
import json  # Para exportar el resumen
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import time
import numpy as np


### INVERSION BRUTA FIJA DE LOS GOBIERNOS NACIONALES POR DEPARTAMENTO 
---
Inicia desde RD16657DA
Termina en RD16682DA 

In [81]:
# ESTA PRIMERA CELDA ES UNA MUESTRA DE LO QUE SE HACE
codigo="RD16682DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija del gobierno nacional por departamentos (millones S/)', 'series': [{'name': 'Inversión bruta fija del gobierno nacional por departamentos (millones S/) - Total (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['3395.695063']}, {'name': '2009', 'values': ['6787.24393901']}, {'name': '2010', 'values': ['9586.56069415']}, {'name': '2011', 'values': ['10130.97789491']}, {'name': '2012', 'values': ['9628.10016668']}]}


##### CODIGO DE SERIES

In [82]:
# GENERAMOS LOS CODIGOS
n_serie=np.linspace(657,682,(683-657))
codigos=[]
for elem in n_serie:
    if elem<10:
        numero='0'+str(int(elem))
    else:
        numero=str(int(elem))

    codigo='RD16'+numero+'DA'
    codigos+=[codigo]

print(codigos)


['RD16657DA', 'RD16658DA', 'RD16659DA', 'RD16660DA', 'RD16661DA', 'RD16662DA', 'RD16663DA', 'RD16664DA', 'RD16665DA', 'RD16666DA', 'RD16667DA', 'RD16668DA', 'RD16669DA', 'RD16670DA', 'RD16671DA', 'RD16672DA', 'RD16673DA', 'RD16674DA', 'RD16675DA', 'RD16676DA', 'RD16677DA', 'RD16678DA', 'RD16679DA', 'RD16680DA', 'RD16681DA', 'RD16682DA']


In [83]:
codigo="RD16708DA" 
url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/2000-01/2024-12' # las fechas de inicio y final tmbn pueden ser delimitadas
response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
data=response.json() # lo almacenamos en un elemenot llamado data
print(data) # la data es un diccionario que contiene todos los valores que necesitamos

{'config': {'title': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/)', 'series': [{'name': 'Inversión bruta fija de gobiernos locales por departamentos (millones S/) - Total (millones de soles)', 'dec': '0'}]}, 'periods': [{'name': '2008', 'values': ['7122.003887']}, {'name': '2009', 'values': ['9324.58241812']}, {'name': '2010', 'values': ['10164.62115863']}, {'name': '2011', 'values': ['8918.52094275']}, {'name': '2012', 'values': ['12790.0212803']}]}


#### BUCLE PARA EXTRAER LA INFORMACIÓN DE TODOS LOS DICCIONARIOS

In [84]:
inicio="2008" # data disponible solo desde 2008
fin="2024" # termina en 2024 todas las series 
dep_nom=[] # para almacenar el nombre de los departamentos
valores=[] # para almacenar las listas que contendran los datos de los departamentos

fechas = pd.date_range(start=inicio, end=fin, freq='YS') # creamos las fechas para que tengan solo mes y año
fechas_lista = fechas.strftime('%Y').tolist() # ACA ESTAN LAS FECHAS

for codigo in codigos:
    url = f'https://estadisticas.bcrp.gob.pe/estadisticas/series/api/{codigo}/json/{inicio}/{fin}' 
    response = requests.get(url) # la respuesta viene a ser el output de la consulta de la dirección dada por el API
    data_1=response.json() # lo almacenamos en un elemento llamado data
    #PRIMERO SE OBTIENEN LOS NOMBRES, que será tmbn el nombre de la columna
    nombre_dep=data_1['config']['series'][0]['name']
    partes = nombre_dep.split(' - ')
    partes_1=partes[1].split(' (')
    nombre_extraido = partes_1[0] # TENER EN MENTE QUE NOMBRE EXTRAIDO SERA EL NOMBRE DE LA COLUMNA
    dep_nom+=[nombre_extraido] # a la lista de nombres que tendremos tmbn como un output
    #SEGUNDO SE OBTIENEN LOS VALORES REGISTRADOS
    series=data_1['periods']
    registros=[elem['values'][0] for elem in series]
    valores+=[registros] # se llena 1 elemento nuevo por cada region
print(dep_nom)
print(len(valores)) # son 25 porque al final contiene los valores totales en formato anual
print(fechas_lista)

['Amazonas', 'Ancash', 'Apurímac', 'Arequipa', 'Ayacucho', 'Cajamarca', 'Callao', 'Cusco', 'Huancavelica', 'Huánuco', 'Ica', 'Junín', 'La Libertad', 'Lambayeque', 'Lima', 'Loreto', 'Madre de Dios', 'Moquegua', 'Pasco', 'Piura', 'Puno', 'San Martín', 'Tacna', 'Tumbes', 'Ucayali', 'Total']
26
['2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019', '2020', '2021', '2022', '2023', '2024']


## TABLA A VALORES NOMINALES

In [85]:
columnas = ['Fecha']+dep_nom
#print(len(columnas))
datos=[fechas_lista]+valores
#print(len(datos))

dic_data={}

for i in range(len(datos)): # se arma el diccionario con la data correspondiente a cada columna
    dic_data[columnas[i]]=datos[i]

#print(len(dic_data))

df=pd.DataFrame(dic_data,columns=columnas)

df = df.apply(pd.to_numeric, errors='coerce')
df 

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,163.204699,132.869047,48.006064,67.410956,131.475199,103.811979,142.575785,179.290627,48.276749,...,95.530595,9.980798,16.950388,109.106257,148.460297,252.788215,19.701460,75.887473,167.062257,3395.695063
1,2009,297.549658,218.398392,113.693567,59.810880,151.460065,178.080534,179.848840,888.622953,47.437451,...,632.460043,9.527151,42.289867,173.473854,255.420302,199.915277,12.842555,127.228433,208.431306,6787.243939
2,2010,335.077632,227.053410,354.986141,48.466612,218.006664,251.837866,599.915702,2037.187949,54.194017,...,221.863890,116.214987,28.160662,237.074232,347.675711,161.069749,17.979016,96.319807,149.903613,9586.560694
3,2011,451.886983,169.868038,167.398506,153.277507,755.366378,632.424917,1214.317535,825.009840,48.569707,...,231.301010,15.037595,10.896468,227.480214,679.415038,94.807373,33.994050,64.647983,86.628181,10130.977895
4,2012,306.918780,164.468490,231.359706,77.667483,586.098618,836.602685,563.401954,730.552697,62.166040,...,246.074242,12.576612,28.066105,130.513839,380.611077,218.700287,16.525872,70.885155,64.090068,9628.100167
5,2013,349.143987,155.951035,374.927080,451.129574,591.168604,861.242988,902.117201,847.225652,69.778714,...,324.502472,17.618008,102.879056,167.487883,666.353795,204.629632,41.642552,77.761655,102.103032,11188.177556
6,2014,303.592493,218.737244,299.595902,604.364409,426.906464,698.207054,1114.937419,795.618617,269.090275,...,224.662139,42.784317,59.993357,320.532204,750.760921,176.416687,75.621029,63.795711,229.879743,13361.829028
7,2015,343.966234,218.721075,253.817759,405.397350,594.903013,769.797843,2249.056915,1050.128044,476.499820,...,247.967737,38.078642,174.048134,653.566185,572.210346,160.606578,46.206663,62.701891,256.165448,15716.894026
8,2016,288.332087,120.596969,162.063859,252.269080,286.576618,397.182327,624.100556,594.755204,391.123628,...,237.866149,45.125503,123.442446,485.869508,783.470572,101.671086,335.453076,44.313715,378.234871,11969.039125
9,2017,341.281381,202.506840,290.749353,339.904042,368.714986,320.268138,690.476314,507.478156,252.167417,...,258.606539,106.556055,126.722605,391.002972,815.477251,339.741532,350.177917,104.475410,329.099284,11942.382227


## TABLA A VALORES REALES

In [86]:
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs'
os.chdir(directorio)
deflactor=pd.read_excel('DEFLACTOR.xlsx')
def_1=deflactor[18:].reset_index(drop=True)
df_real=df.copy()
#print(def_1)
#print(df.dtypes)
#deflactor['DEFLACTOR'] = pd.to_numeric(deflactor['DEFLACTOR'], errors='coerce')
df_real.iloc[:, 1:] = df_real.iloc[:, 1:].div(def_1['DEFLACTOR'], axis=0)
df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
df_real

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,160.464501,130.638183,47.200045,66.279130,129.267737,102.068981,140.181945,176.280347,47.466185,...,93.926641,9.813221,16.665792,107.274369,145.967656,248.543914,19.370674,74.613326,164.257290,3338.681520
1,2009,285.908917,209.854208,109.245646,57.470958,145.534643,171.113665,172.812792,853.858237,45.581602,...,607.716935,9.154430,40.635402,166.687208,245.427746,192.094189,12.340128,122.251001,200.277054,6521.713310
2,2010,303.681791,205.779138,321.724928,43.925425,197.580047,228.241359,543.705272,1846.309113,49.116188,...,201.075861,105.325966,25.522087,214.861036,315.099466,145.977962,16.294432,87.294910,135.858062,8688.326660
3,2011,388.956406,146.211916,144.086296,131.931812,650.172726,544.352310,1045.209537,710.117518,41.805804,...,199.089624,12.943433,9.379007,195.800919,584.798504,81.604332,29.259979,55.644991,74.564188,8720.120077
4,2012,259.619307,139.122133,195.704696,65.698092,495.774540,707.673246,476.575675,617.966697,52.585587,...,208.151564,10.638421,23.740817,110.400258,321.954831,184.996229,13.979058,59.960993,54.213102,8144.306751
5,2013,290.436573,129.728381,311.884324,375.273619,491.765546,716.427810,750.429159,704.767443,58.045653,...,269.938448,14.655598,85.580281,139.325346,554.308595,170.221832,34.640493,64.686289,84.934743,9306.922276
6,2014,245.242624,176.696385,242.014170,488.206781,344.855897,564.013058,900.648682,642.702314,217.371664,...,181.482526,34.561257,48.462754,258.926557,606.466177,142.509754,61.086819,51.534304,185.697317,10793.712279
7,2015,270.192064,171.809593,199.378711,318.447383,467.307709,604.690948,1766.677278,824.895689,374.299734,...,194.783406,29.911502,136.718143,513.388754,449.482185,126.159543,36.296218,49.253536,201.222865,12345.921252
8,2016,218.202216,91.264646,122.645709,190.910671,216.873723,300.577244,472.303051,450.095253,295.992178,...,180.010908,34.149806,93.418029,367.693393,592.910130,76.942031,253.862154,33.535466,286.238302,9057.857173
9,2017,248.490267,147.447184,211.697410,247.487413,268.464939,233.190321,502.742466,369.499742,183.605529,...,188.293916,77.584492,92.267952,284.693037,593.756855,247.369088,254.967921,76.069554,239.620364,8695.363735


## TABLA A VALORES PORCENTUALES

In [87]:
df_por=df_real.copy()
#print(def_1)
df_por.iloc[:, 1:] = df_por.iloc[:, 1:].div(def_1['PBI_real'], axis=0)
#df_real.iloc[:, 1:] = df_real.iloc[:, 1:] * 100
directorio='C:/Users/FERNANDO/Documents/PI_INEQUIDAD/scripts_data/data/outputs/inversion bruta/gob_centralxdep'
os.chdir(directorio)
df_por.to_excel('inv_bruta_gcentral.xlsx', index=False)
df_real.to_excel('ureales_inv_bruta_gcentrals.xlsx',index=False)
df.to_excel('unominales_inv_bruta_gcentrals.xlsx',index=False)
df_por

,Fecha,Amazonas,Ancash,Apurímac,Arequipa,Ayacucho,Cajamarca,Callao,Cusco,Huancavelica,...,Madre de Dios,Moquegua,Pasco,Piura,Puno,San Martín,Tacna,Tumbes,Ucayali,Total
0,2008,0.000460,0.000374,0.000135,0.000190,0.000371,0.000293,0.000402,0.000505,0.000136,...,0.000269,0.000028,0.000048,0.000307,0.000418,0.000712,0.000056,0.000214,0.000471,0.009570
1,2009,0.000811,0.000595,0.000310,0.000163,0.000413,0.000485,0.000490,0.002421,0.000129,...,0.001723,0.000026,0.000115,0.000473,0.000696,0.000545,0.000035,0.000347,0.000568,0.018491
2,2010,0.000795,0.000539,0.000842,0.000115,0.000517,0.000597,0.001423,0.004832,0.000129,...,0.000526,0.000276,0.000067,0.000562,0.000825,0.000382,0.000043,0.000228,0.000356,0.022739
3,2011,0.000957,0.000360,0.000355,0.000325,0.001600,0.001340,0.002573,0.001748,0.000103,...,0.000490,0.000032,0.000023,0.000482,0.001439,0.000201,0.000072,0.000137,0.000184,0.021465
4,2012,0.000602,0.000323,0.000454,0.000152,0.001150,0.001641,0.001105,0.001433,0.000122,...,0.000483,0.000025,0.000055,0.000256,0.000747,0.000429,0.000032,0.000139,0.000126,0.018888
5,2013,0.000636,0.000284,0.000683,0.000822,0.001077,0.001570,0.001644,0.001544,0.000127,...,0.000591,0.000032,0.000187,0.000305,0.001214,0.000373,0.000076,0.000142,0.000186,0.020390
6,2014,0.000525,0.000378,0.000518,0.001045,0.000738,0.001207,0.001927,0.001375,0.000465,...,0.000388,0.000074,0.000104,0.000554,0.001298,0.000305,0.000131,0.000110,0.000397,0.023098
7,2015,0.000560,0.000356,0.000413,0.000660,0.000969,0.001253,0.003661,0.001710,0.000776,...,0.000404,0.000062,0.000283,0.001064,0.000932,0.000261,0.000075,0.000102,0.000417,0.025587
8,2016,0.000435,0.000182,0.000245,0.000381,0.000432,0.000599,0.000942,0.000897,0.000590,...,0.000359,0.000068,0.000186,0.000733,0.001182,0.000153,0.000506,0.000067,0.000571,0.018059
9,2017,0.000483,0.000287,0.000412,0.000481,0.000522,0.000453,0.000978,0.000719,0.000357,...,0.000366,0.000151,0.000179,0.000554,0.001155,0.000481,0.000496,0.000148,0.000466,0.016910
